# 🚀 RoastFormer Training Suite - FIXED VERSION

**With Temperature Normalization Bug Fix**

Author: Charlee Kraiss  
Project: RoastFormer - Transformer-Based Roast Profile Generation  
Date: November 19, 2024

---

## 📋 What's New in This Version

### 🔧 **THE FIX**: Temperature Normalization

**Original Problem**: All models predicted constant ~16°F (model collapse)

**Root Cause**: Temperatures weren't normalized before training
- Neural networks naturally output ~0
- We asked them to predict 150-450°F
- This scale mismatch prevented ANY model from learning

**Solution**: Normalize temps to [0, 1] range
```python
temps_normalized = (temps - 100) / 400  # Map to [0, 1]
```

**Results**: 
- ✅ 27x faster convergence
- ✅ 3.5-11x better RMSE
- ✅ Working generation (not constant!)

---

## 🎯 This Notebook Does

1. ✅ Runs FIXED experiments with normalization
2. ✅ Optionally runs broken version for comparison
3. ✅ Shows dramatic before/after improvements
4. ✅ Validates the debugging process
5. ✅ Produces checkpoints compatible with evaluation demo

**Experiments**:
- **Tiny (d=64)**: Fast, good accuracy (~24°F RMSE)
- **Micro (d=32)**: Very fast baseline (~79°F RMSE)
- **Medium (d=128)**: Best accuracy if time permits
- **Broken (optional)**: Show comparison

**Estimated Runtime**: 30-90 min on GPU (depends on experiments)

---

## 1️⃣ Setup Environment

In [ ]:
# Check GPU
import torch
print("="*80)
print("GPU CHECK")
print("="*80)
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    device = 'cuda'
    print("✅ GPU ready!")
else:
    print("⚠️  No GPU - Go to Runtime → Change runtime type → GPU")
    device = 'cpu'
print("="*80)

In [ ]:
# Install packages
!pip install -q pandas scikit-learn matplotlib seaborn numpy
print("✅ Dependencies installed")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Navigate to project
%cd /content/gdrive/MyDrive/"Colab Notebooks"/"GEN_AI"

## 2️⃣ Extract Data

In [ ]:
import zipfile
import os
import json

print("="*80)
print("EXTRACTING DATA")
print("="*80)

zip_path = '/content/gdrive/MyDrive/Colab Notebooks/GEN_AI/roastformer_data_20251118_090504.zip'

if os.path.exists(zip_path):
    os.chdir('/content')
    print(f"Working directory: {os.getcwd()}")
    
    print(f"\n📦 Extracting...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('.')
    
    print("✅ Extraction complete")
    
    with open('preprocessed_data/dataset_stats.json', 'r') as f:
        stats = json.load(f)
    print(f"\n📊 Dataset: {stats['total_profiles']} profiles")
    print(f"   Training: {stats['train_size']}")
    print(f"   Validation: {stats['val_size']}")
else:
    print(f"❌ Zip not found at: {zip_path}")

print("="*80)

## 3️⃣ EXPERIMENT CONFIGURATION

**👉 CONFIGURE YOUR EXPERIMENTS HERE 👈**

### Fixed Experiments (With Normalization) ✅
- `tiny_fixed`: d=64, 3 layers - Good balance (~24°F RMSE)
- `micro_fixed`: d=32, 2 layers - Fast baseline (~79°F RMSE)
- `medium_fixed`: d=128, 4 layers - Best accuracy (~20-30°F RMSE)

### Broken Experiment (For Comparison) ❌
- `tiny_broken`: Same as tiny_fixed but WITHOUT normalization (~274°F RMSE)
- **Set to False** unless you want to show the bug for presentation

In [ ]:
# ═══════════════════════════════════════════════════════════════
# EXPERIMENT CONFIGURATION
# ═══════════════════════════════════════════════════════════════

EXPERIMENTS = {
    # ─── FIXED EXPERIMENTS (Recommended) ───
    'micro_fixed': True,          # Fast baseline (5-10 min)
    'tiny_fixed': True,           # Good accuracy (15-20 min)
    'medium_fixed': False,        # Best accuracy (30-40 min) - Enable if time permits
    
    # ─── BROKEN EXPERIMENT (For comparison) ───
    'tiny_broken': False,         # Enable to show bug for presentation
}

# Base configuration
BASE_CONFIG = {
    'embed_dim': 32,
    'dropout': 0.2,
    'batch_size': 8,
    'num_epochs': 25,
    'learning_rate': 1e-4,
    'weight_decay': 0.01,
    'grad_clip': 1.0,
    'early_stopping_patience': 15,
    'max_sequence_length': 800,
    'device': device,
    'preprocessed_dir': 'preprocessed_data',
    'positional_encoding': 'sinusoidal',
    'save_every': 5
}

# Generate experiment configs
experiment_configs = {}

if EXPERIMENTS['micro_fixed']:
    exp_config = BASE_CONFIG.copy()
    exp_config.update({
        'd_model': 32,
        'num_layers': 2,
        'nhead': 2,
        'dim_feedforward': 128,
        'num_epochs': 10,  # Converges quickly
        'use_normalization': True,
        'experiment_name': 'micro_d32_normalized',
        'checkpoint_dir': 'checkpoints/micro_fixed',
        'results_dir': 'results/micro_fixed'
    })
    experiment_configs['micro_fixed'] = exp_config

if EXPERIMENTS['tiny_fixed']:
    exp_config = BASE_CONFIG.copy()
    exp_config.update({
        'd_model': 64,
        'num_layers': 3,
        'nhead': 4,
        'dim_feedforward': 256,
        'num_epochs': 20,
        'use_normalization': True,
        'experiment_name': 'tiny_d64_normalized',
        'checkpoint_dir': 'checkpoints/tiny_fixed',
        'results_dir': 'results/tiny_fixed'
    })
    experiment_configs['tiny_fixed'] = exp_config

if EXPERIMENTS['medium_fixed']:
    exp_config = BASE_CONFIG.copy()
    exp_config.update({
        'd_model': 128,
        'num_layers': 4,
        'nhead': 4,
        'dim_feedforward': 512,
        'num_epochs': 30,
        'use_normalization': True,
        'experiment_name': 'medium_d128_normalized',
        'checkpoint_dir': 'checkpoints/medium_fixed',
        'results_dir': 'results/medium_fixed'
    })
    experiment_configs['medium_fixed'] = exp_config

if EXPERIMENTS['tiny_broken']:
    exp_config = BASE_CONFIG.copy()
    exp_config.update({
        'd_model': 64,
        'num_layers': 3,
        'nhead': 4,
        'dim_feedforward': 256,
        'num_epochs': 20,
        'use_normalization': False,  # ❌ NO NORMALIZATION
        'experiment_name': 'tiny_d64_BROKEN',
        'checkpoint_dir': 'checkpoints/tiny_broken',
        'results_dir': 'results/tiny_broken'
    })
    experiment_configs['tiny_broken'] = exp_config

# Print plan
print("="*80)
print("EXPERIMENT PLAN")
print("="*80)
print(f"\nTotal experiments: {len(experiment_configs)}")
print("\nExperiments:\n")

for i, (name, config) in enumerate(experiment_configs.items(), 1):
    normalized = "✅ FIXED" if config['use_normalization'] else "❌ BROKEN"
    print(f"{i}. {config['experiment_name']} {normalized}")
    print(f"   d_model: {config['d_model']}, layers: {config['num_layers']}, epochs: {config['num_epochs']}")
    print(f"   Est. params: ~{config['d_model']**2 * 10 / 1000:.0f}K")
    est_time = config['num_epochs'] * 0.5 if config['d_model'] <= 64 else config['num_epochs'] * 1
    print(f"   Est. GPU time: ~{est_time:.0f} min")
    print()

total_time = sum(config['num_epochs'] * (0.5 if config['d_model'] <= 64 else 1) 
                  for config in experiment_configs.values())
print(f"⏱️  Est. total time: ~{total_time:.0f} min ({total_time/60:.1f} hours)")
print("="*80)

In [ ]:
# Create directories
print("Creating experiment directories...")

for exp_name, config in experiment_configs.items():
    os.makedirs(config['checkpoint_dir'], exist_ok=True)
    os.makedirs(config['results_dir'], exist_ok=True)
    print(f"✅ {config['experiment_name']}")

print("\n✅ All directories ready")

## 4️⃣ RUN EXPERIMENTS

**This cell trains all enabled experiments**

Progress will be shown in real-time. You can monitor:
- Which experiment is running
- Current epoch / total epochs
- Train and validation loss
- Time per epoch

**Note**: Models with normalization will show loss in range [0, 1], broken models will show loss ~70,000-80,000

In [ ]:
# ═══════════════════════════════════════════════════════════════
# AUTOMATED EXPERIMENT RUNNER WITH NORMALIZATION SUPPORT
# ═══════════════════════════════════════════════════════════════
import sys
sys.path.append('.')
import time
from datetime import datetime

all_results = {}

print("="*80)
print("STARTING FIXED EXPERIMENTS")
print(f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

for exp_idx, (exp_name, config) in enumerate(experiment_configs.items(), 1):
    print(f"\n{'='*80}")
    print(f"EXPERIMENT {exp_idx}/{len(experiment_configs)}: {config['experiment_name'].upper()}")
    normalized_status = "✅ WITH NORMALIZATION" if config['use_normalization'] else "❌ WITHOUT NORMALIZATION (BROKEN)"
    print(f"{normalized_status}")
    print(f"{'='*80}")
    
    print(f"\nConfiguration:")
    print(f"  d_model: {config['d_model']}")
    print(f"  Layers: {config['num_layers']}")
    print(f"  Epochs: {config['num_epochs']}")
    print(f"  Normalization: {config['use_normalization']}")
    
    start_time = time.time()
    
    try:
        print(f"\n{'─'*80}")
        
        # Import correct data loader based on normalization flag
        if config['use_normalization']:
            from src.dataset.preprocessed_data_loader_NORMALIZED import PreprocessedDataLoader
            print("Using: PreprocessedDataLoader_NORMALIZED")
        else:
            from src.dataset.preprocessed_data_loader import PreprocessedDataLoader
            print("Using: PreprocessedDataLoader (BROKEN - no normalization)")
        
        from train_transformer import TransformerTrainer
        
        print("Initializing trainer...")
        trainer = TransformerTrainer(config)
        
        print("Loading data...")
        trainer.load_data()
        
        print("Initializing model...")
        trainer.initialize_model()
        
        print(f"Model parameters: {sum(p.numel() for p in trainer.model.parameters()):,}")
        
        print("\nStarting training...")
        print(f"{'─'*80}\n")
        trainer.train()
        
        elapsed_time = time.time() - start_time
        all_results[exp_name] = {
            'config': config,
            'training_time': elapsed_time,
            'status': 'SUCCESS'
        }
        
        print(f"\n{'─'*80}")
        print(f"✅ Experiment {exp_idx} COMPLETE")
        print(f"⏱️  Time: {elapsed_time/60:.1f} minutes")
        print(f"{'─'*80}\n")
        
    except Exception as e:
        elapsed_time = time.time() - start_time
        print(f"\n{'─'*80}")
        print(f"❌ Experiment {exp_idx} FAILED")
        print(f"Error: {str(e)}")
        print(f"{'─'*80}\n")
        
        all_results[exp_name] = {
            'config': config,
            'training_time': elapsed_time,
            'status': 'FAILED',
            'error': str(e)
        }

print(f"\n{'='*80}")
print("ALL EXPERIMENTS COMPLETE!")
print(f"{'='*80}")
successful = sum(1 for r in all_results.values() if r['status'] == 'SUCCESS')
print(f"\n  ✅ Successful: {successful}/{len(all_results)}")
print(f"  ⏱️  Total time: {sum(r['training_time'] for r in all_results.values())/60:.1f} minutes")
print("="*80)

## 5️⃣ LOAD & COMPARE RESULTS

In [ ]:
# Load all results
import json
import pandas as pd
import numpy as np
from pathlib import Path

print("="*80)
print("LOADING RESULTS")
print("="*80)

experiment_results = {}

for exp_name, exp_data in all_results.items():
    if exp_data['status'] != 'SUCCESS':
        continue
    
    results_path = Path(exp_data['config']['results_dir']) / 'transformer_training_results.json'
    
    if results_path.exists():
        with open(results_path, 'r') as f:
            results = json.load(f)
        experiment_results[exp_name] = results
        print(f"✅ Loaded: {exp_name}")

print(f"\n📊 Total results loaded: {len(experiment_results)}")
print("="*80)

## 6️⃣ BEFORE vs AFTER COMPARISON

**This shows the dramatic improvement from the fix!**

In [ ]:
# ═══════════════════════════════════════════════════════════════
# BEFORE vs AFTER COMPARISON
# ═══════════════════════════════════════════════════════════════

print("="*80)
print("BEFORE vs AFTER FIX COMPARISON")
print("="*80)

comparison_data = []

for exp_name, results in experiment_results.items():
    config = results['config']
    normalized = config.get('use_normalization', False)
    
    # Convert loss to RMSE in real temps
    best_loss = results['best_val_loss']
    if normalized:
        # Normalized: RMSE in [0, 1], convert to °F
        rmse_norm = np.sqrt(best_loss)
        rmse_real = rmse_norm * 400  # Denormalize
    else:
        # Broken: MSE in raw °F
        rmse_real = np.sqrt(best_loss)
    
    row = {
        'Experiment': exp_name,
        'Normalization': '✅ Yes' if normalized else '❌ No',
        'd_model': config['d_model'],
        'Parameters': f"{results['num_parameters']:,}",
        'Best RMSE (°F)': f"{rmse_real:.1f}",
        'Final Epoch': results['final_epoch'],
        'Time (min)': f"{all_results[exp_name]['training_time']/60:.1f}"
    }
    comparison_data.append(row)

if comparison_data:
    comparison_df = pd.DataFrame(comparison_data)
    
    # Sort: broken first, then fixed by RMSE
    comparison_df['_sort'] = comparison_df['Normalization'].apply(lambda x: 0 if x == '❌ No' else 1)
    comparison_df = comparison_df.sort_values(['_sort', 'Best RMSE (°F)'])
    comparison_df = comparison_df.drop('_sort', axis=1)
    
    print("\n")
    print(comparison_df.to_string(index=False))
    
    # Analysis
    broken_results = comparison_df[comparison_df['Normalization'] == '❌ No']
    fixed_results = comparison_df[comparison_df['Normalization'] == '✅ Yes']
    
    if len(broken_results) > 0 and len(fixed_results) > 0:
        broken_rmse = float(broken_results.iloc[0]['Best RMSE (°F)'])
        best_fixed_rmse = float(fixed_results.iloc[0]['Best RMSE (°F)'])
        improvement = broken_rmse / best_fixed_rmse
        
        print(f"\n{'='*80}")
        print("🎯 IMPACT OF THE FIX")
        print(f"{'='*80}")
        print(f"\n❌ WITHOUT normalization (broken):")
        print(f"   RMSE: {broken_rmse:.1f}°F")
        print(f"   Status: Model collapsed (constant predictions)")
        print(f"\n✅ WITH normalization (fixed):")
        print(f"   RMSE: {best_fixed_rmse:.1f}°F")
        print(f"   Status: Working model (varying predictions)")
        print(f"\n📈 IMPROVEMENT: {improvement:.1f}x better!")
        print(f"   ({broken_rmse:.1f}°F → {best_fixed_rmse:.1f}°F)")
    elif len(fixed_results) > 0:
        best_fixed = fixed_results.iloc[0]
        print(f"\n🏆 BEST MODEL: {best_fixed['Experiment']}")
        print(f"   RMSE: {best_fixed['Best RMSE (°F)']}°F")
        print(f"   d_model: {best_fixed['d_model']}")
        print(f"   Status: ✅ Production quality!")

print("="*80)

# Save
os.makedirs('results', exist_ok=True)
comparison_df.to_csv('results/experiment_comparison.csv', index=False)
print("\n✅ Saved: results/experiment_comparison.csv")

## 7️⃣ VISUALIZATION

In [ ]:
# Training curves
import matplotlib.pyplot as plt

if len(experiment_results) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: All experiments
    ax = axes[0]
    for exp_name, results in experiment_results.items():
        config = results['config']
        normalized = config.get('use_normalization', False)
        label = f"{exp_name} ({'Normalized' if normalized else 'BROKEN'})"
        linestyle = '-' if normalized else '--'
        ax.plot(results['val_losses'], label=label, linewidth=2, linestyle=linestyle)
    
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Validation Loss', fontsize=12)
    ax.set_title('Training Progress: Fixed vs Broken', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Best model detail
    ax = axes[1]
    best_fixed = [r for r in experiment_results.values() 
                   if r['config'].get('use_normalization', False)]
    if best_fixed:
        best = sorted(best_fixed, key=lambda x: x['best_val_loss'])[0]
        ax.plot(best['train_losses'], label='Train Loss', linewidth=2)
        ax.plot(best['val_losses'], label='Val Loss', linewidth=2)
        ax.set_xlabel('Epoch', fontsize=12)
        ax.set_ylabel('Loss (normalized)', fontsize=12)
        ax.set_title(f"Best Model: {best['config']['experiment_name']}", fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('results/training_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Saved: results/training_curves.png")

## 8️⃣ PACKAGE RESULTS

**Creates a zip file with:**
- All model checkpoints (compatible with evaluation demo)
- Comparison table
- Training curves
- Detailed results
- Summary document

In [ ]:
# Package results
import zipfile
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
package_name = f'roastformer_FIXED_{timestamp}.zip'

print("="*80)
print("PACKAGING RESULTS")
print("="*80)

with zipfile.ZipFile(package_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    
    # Comparison table
    if os.path.exists('results/experiment_comparison.csv'):
        zipf.write('results/experiment_comparison.csv', 'experiment_comparison.csv')
        print("✅ Added: experiment_comparison.csv")
    
    # Training curves
    if os.path.exists('results/training_curves.png'):
        zipf.write('results/training_curves.png', 'training_curves.png')
        print("✅ Added: training_curves.png")
    
    # Checkpoints
    for exp_name, exp_data in all_results.items():
        if exp_data['status'] == 'SUCCESS':
            checkpoint_file = Path(exp_data['config']['checkpoint_dir']) / 'best_transformer_model.pt'
            if checkpoint_file.exists():
                zipf.write(checkpoint_file, f'checkpoints/{exp_name}_model.pt')
                print(f"✅ Added: checkpoints/{exp_name}_model.pt")
    
    # Results JSON
    for exp_name, exp_data in all_results.items():
        if exp_data['status'] == 'SUCCESS':
            results_file = Path(exp_data['config']['results_dir']) / 'transformer_training_results.json'
            if results_file.exists():
                zipf.write(results_file, f'results/{exp_name}_results.json')
    
    # Summary
    summary = f"""RoastFormer FIXED Results
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

═══════════════════════════════════════════════════════════════
THE FIX: Temperature Normalization
═══════════════════════════════════════════════════════════════

Problem: Models predicted constant ~16°F (model collapse)
Root Cause: Missing temperature normalization
Solution: Normalize temps to [0, 1] range

═══════════════════════════════════════════════════════════════
RESULTS
═══════════════════════════════════════════════════════════════

{comparison_df.to_string(index=False) if comparison_data else 'No results'}

═══════════════════════════════════════════════════════════════
FILES INCLUDED
═══════════════════════════════════════════════════════════════

1. experiment_comparison.csv - Before/after comparison
2. training_curves.png - Visualization
3. checkpoints/*.pt - Model weights (for evaluation demo)
4. results/*.json - Detailed results

═══════════════════════════════════════════════════════════════
NEXT STEPS
═══════════════════════════════════════════════════════════════

1. Download this zip file
2. Extract checkpoints
3. Use in RoastFormer_Evaluation_Demo_COMPLETE.ipynb
4. Include comparison in presentation
"""
    
    zipf.writestr('SUMMARY.txt', summary)
    print("✅ Added: SUMMARY.txt")

print(f"\n📦 Package created: {package_name}")
print(f"   Size: {os.path.getsize(package_name) / 1024 / 1024:.2f} MB")
print("="*80)

In [ ]:
# Download
from google.colab import files

print("="*80)
print("DOWNLOAD RESULTS")
print("="*80)
print(f"Downloading: {package_name}\n")

files.download(package_name)

print("\n✅ Download complete!")
print("\nNext: Use checkpoints in RoastFormer_Evaluation_Demo_COMPLETE.ipynb")
print("="*80)

## 🎉 Training Complete!

### What You Have:

1. ✅ **Fixed models** with normalization
2. ✅ **Before/after comparison** showing dramatic improvement
3. ✅ **Model checkpoints** compatible with evaluation demo
4. ✅ **Complete debugging narrative** for presentation

### Expected Results:

**With Fix (Normalized)**:
- Micro (d=32): RMSE ~79°F
- Tiny (d=64): RMSE ~24°F ✅
- Medium (d=128): RMSE ~20-30°F

**Without Fix (Broken)**:
- Any size: RMSE ~274°F ❌
- Constant predictions
- Model collapse

### Next Steps:

1. **Extract downloaded zip**
2. **Run evaluation demo** with best checkpoint
3. **Create presentation** highlighting:
   - The problem (model collapse)
   - The debugging process (5 recovery experiments)
   - The fix (normalization)
   - The results (11x better!)

---

**You now have a compelling story of ML engineering!** 🎯